### Fourier Transform Speed Up

The Fourier transform algorithm used in both OpenCV and Numpy is the Fast Fourier Transform. I will not go into details, but compared to the naive discrete fourier transform algorithm (DFT), the FFT algorithm is much faster, specially as the size of the image increases.

Although FFT is proven to be more efficient than naive DFT, it is still much faster for array sizes of powers of two, three and five. This is due to the underlying symmetry of the FT calculation for real values; the FFT algorithm speeds up the DFT calculation by taking advantage of the complex conjugate symmetry of the FT if the input function is real. 

#### Padding Up the Image

Theoretically, the FFT algorithm would perform optimally if, given an image $I$ the dimensions of the image $(h,w)$ are both powers of two, three or five, i.e $(h,w)=(2^{y_1}3^{y_2}5^{y_3}, 2^{x_1}3^{x_2}5^{x_3})$ for integers $x_i,y_i$. Therefore, the goal is to force the contents of the image into an array of shape $(2^{y_1}3^{y_2}5^{y_3}, 2^{x_1}3^{x_2}5^{x_3})$ by padding the image with zeros on the bottom and right. This zero-padding will not change the contents of the amplitude spectrum of the image, but may affect the phase.

The cell below will implement a method of "forcing" the image into the optimal array size by padding it with zeros.

In [78]:
%matplotlib inline
from matplotlib import pyplot as plt
import utilities as ut
import numpy as np
import cv2

# Read the image to analyse
image = ut.read_image("images/Lenna.png", "BGR2GRAY")

def optimal_size(n):
    """
    This function will return the next
    power of two closest to n.
    """
    
    # OpenCV function to determine the optimal
    # DFT size. It returns r=(2^p)*(3^q)*(5^z)
    # where n<=r. 
    return cv2.getOptimalDFTSize(n)

# Get the number
rows, cols = image.shape

# Get the new image size 
nrows, ncols = optimal_size(rows), optimal_size(cols)

print("Old dimension: (%d,%d)" % (rows,cols))
print("New dimension: (%d,%d)" % (nrows,ncols))

Old dimension: (566,1070)
New dimension: (576,1080)


#### Comparing the FFT Calculations 

In the next cell, different methods for calculating the DFT of an image will be compared. The numpy FFT package and OpenCV's own FFT will be compared, using the normal image and the padded image. 

In [79]:
def cv_fourier(gray_image):
    """
    FFT calculation with OpenCV's
    dft function.
    """
    
    # Calculate FFT (amplitude and phase)
    dft = cv2.dft(np.float32(image), flags= cv2.DFT_COMPLEX_OUTPUT)

    # Shift DFT so frequency 0 is at center
    return np.fft.fftshift(dft)

def np_fourier(grey_image):
    """
    FFT calculation with Numpy's
    fft package.
    """
    
    # Calculate FFT (amplitude and phase)
    dft = np.fft.fft2(grey_image)

    # Shift DFT so frequency 0 is at center
    return np.fft.fftshift(dft)

# Get the padded image
new_image = np.zeros((nrows, ncols))
new_image[:rows, :cols] = image

# Time the calculations
%timeit cv_fourier(image) # Cv fourier with original size image
%timeit np_fourier(image) # Np fourier with original size image

%timeit cv_fourier(new_image) # Cv fourier with optimal image
%timeit np_fourier(new_image) # Np fourier with optimal image

The slowest run took 8.82 times longer than the fastest. This could mean that an intermediate result is being cached 
10 loops, best of 3: 22.2 ms per loop
1 loops, best of 3: 218 ms per loop
10 loops, best of 3: 22.5 ms per loop
10 loops, best of 3: 48.1 ms per loop


From the results above, it can be seen that the OpenCV's FFT outdoes the Numpy's implementation in terms of performance. Although it is reccommended to pad the images with zeros for the performance increase, there has not been a huge improvement in the test done above.

With all of this in mind, the fourier transform method used in the file `analysis.py` will be the OpenCV's version, together with the zero padding.